## Data Extraction

In [1]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv


Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


## Extract the zip file

In [2]:
!powershell -Command "Expand-Archive -Path 'C:\Users\Admin\Documents\Guvi\MDTM38\project\retail_order\orders.csv.zip' -DestinationPath 'C:\Users\Admin\Documents\Guvi\MDTM38\project\1'"

## Data Cleaning and Spliting the data into two

In [ ]:
import pandas as pd
import pymysql


# import the data from csv file
df=pd.read_csv("C://Users//Admin//Documents//Guvi//MDTM38//project//retail_order//orders.csv")

# Arranging column name 
df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)

# Drop na and filling with 0
df=df[~df["ship_mode"].isin(["Not Available","unknown"])]
df = df.dropna(subset=['order_id','quantity','ship_mode'])
numerical_cols = ['cost_price', 'list_price', 'quantity','discount_percent']
df[numerical_cols] = df[numerical_cols].fillna(0)

# addding new column discount,sale_price,profit

df['discount'] = df['list_price'] * df['discount_percent'] / 100
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = (df['sale_price'] - df['cost_price']) * df['quantity']

# Spliting the dataframe into two dataframe
# DF-1 10-columns 

df_first = df.iloc[:, :9]
df_first.to_csv('df1.csv', index=False)

# DF-2 remaining columns (10) + order_id 

df_second = df.iloc[:, 9:]
df_second = pd.concat([df[['order_id']], df_second], axis=1)
df_second.to_csv('df2.csv', index=False)

 
##   connection python with mysql


myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='11111')

# creating a new schema if not exists 
myconnection.cursor().execute("Create database if not exists retail_orders")

# use the schema for querying 
myconnection.cursor().execute("use retail_orders")

# outer join df1 + df2 (create table as df3)
myconnection.cursor().execute("""CREATE TABLE IF NOT EXISTS df3 AS
SELECT COALESCE(df1.order_id, df2.order_id) AS order_id, df1.order_date, df1.ship_mode, df1.segment, 
df1.country, df1.city, df1.state, df1.postal_code, df1.region, 
df2.category, df2.sub_category, df2.product_id, df2.cost_price, df2.list_price, df2.quantity, 
df2.discount_percent, df2.discount, df2.sale_price, df2.profit
FROM df1
LEFT JOIN df2 ON df1.order_id = df2.order_id
UNION
SELECT COALESCE(df1.order_id, df2.order_id) AS order_id, df1.order_date, df1.ship_mode, df1.segment, 
df1.country, df1.city, df1.state, df1.postal_code, df1.region, 
df2.category, df2.sub_category, df2.product_id, df2.cost_price, df2.list_price, df2.quantity, 
df2.discount_percent, df2.discount, df2.sale_price, df2.profit
FROM df1
RIGHT JOIN df2 ON df1.order_id = df2.order_id;""")



In [3]:
import pandas as pd
df=pd.read_csv("C://Users//Admin//Documents//Guvi//MDTM38//project//retail_order//orders.csv")
df.head(5)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [4]:
## Arranging column name 

df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'cost_price', 'list_price', 'quantity',
       'discount_percent'],
      dtype='object')

In [5]:
## Drop na and filling with 0
   
    
df=df[~df["ship_mode"].isin(["Not Available","unknown"])]
df = df.dropna(subset=['order_id','quantity','ship_mode'])
numerical_cols = ['cost_price', 'list_price', 'quantity','discount_percent']
df[numerical_cols] = df[numerical_cols].fillna(0)

In [6]:
## addding new column discount,sale_price,profit

df['discount'] = df['list_price'] * df['discount_percent'] / 100
df['sale_price'] = df['list_price'] - df['discount']
df['profit'] = (df['sale_price'] - df['cost_price']) * df['quantity']
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,29.6
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,324.3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-1.0
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,804.0
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-2.0


In [7]:
## Spliting the dataframe into two dataframe
## DF-1 10-columns 

df_first = df.iloc[:, :9]
df_first.to_csv('df1.csv', index=False)
df_first.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South


In [8]:
## DF-2 remaining columns (10) + order_id 

df_second = df.iloc[:, 9:]
df_second = pd.concat([df[['order_id']], df_second], axis=1)
df_second.to_csv('df2.csv', index=False)
df_second.head(5)

,order_id,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,29.6
1,2,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,324.3
2,3,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-1.0
3,4,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,804.0
4,5,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-2.0


## Connecting python with mysql

In [9]:
import pandas as pd
import pymysql

# connection python with mysql
myconnection = pymysql.connect(host='127.0.0.1',user='root',passwd='11111')

# creating a new schema if not exists 
myconnection.cursor().execute("Create database if not exists retail_orders")

# use the schema for querying 
myconnection.cursor().execute("use retail_orders")

# outer join df1 + df2 (create table as df3)
myconnection.cursor().execute("""CREATE TABLE IF NOT EXISTS df3 AS
SELECT COALESCE(df1.order_id, df2.order_id) AS order_id, df1.order_date, df1.ship_mode, df1.segment, 
df1.country, df1.city, df1.state, df1.postal_code, df1.region, 
df2.category, df2.sub_category, df2.product_id, df2.cost_price, df2.list_price, df2.quantity, 
df2.discount_percent, df2.discount, df2.sale_price, df2.profit
FROM df1
LEFT JOIN df2 ON df1.order_id = df2.order_id
UNION
SELECT COALESCE(df1.order_id, df2.order_id) AS order_id, df1.order_date, df1.ship_mode, df1.segment, 
df1.country, df1.city, df1.state, df1.postal_code, df1.region, 
df2.category, df2.sub_category, df2.product_id, df2.cost_price, df2.list_price, df2.quantity, 
df2.discount_percent, df2.discount, df2.sale_price, df2.profit
FROM df1
RIGHT JOIN df2 ON df1.order_id = df2.order_id;""")



0

## Questions

In [ ]:
# 1. Find top 10 highest revenue generating products.
# Identifies best-selling products that drive the most revenue.

answer_1 = pd.read_sql_query("""SELECT category,sub_category,product_id,  
ROUND(SUM(sale_price * quantity), 2) AS revenue FROM df2
GROUP BY category, sub_category, product_id
ORDER BY revenue DESC
LIMIT 10; """,myconnection)

# 2. Find the top 5 cities with the highest profit margins.
# Helps understand which cities yield the highest profit margins.

answer_2 = pd.read_sql_query("""SELECT city, 
ROUND((SUM(profit) / SUM(sale_price * quantity)) * 100, 2) AS profit_margin FROM df3
GROUP BY city
ORDER BY profit_margin DESC
LIMIT 5;
""",myconnection)

# 3. Calculate the total discount given for each category.
# Reveals which product categories receive the highest total discounts.

answer_3= pd.read_sql_query(""" SELECT category, 
ROUND(SUM(discount),2) AS total_discount FROM df3
GROUP BY category
ORDER BY total_discount DESC;
;
 """,myconnection)

# 4. Find the average sale price per product category.
# Helps in pricing strategy and competitive analysis.

answer_4= pd.read_sql_query(""" SELECT category, 
ROUND(AVG(sale_price),2) AS avg_sale_price FROM df3
GROUP BY category
ORDER BY avg_sale_price DESC;
""",myconnection)

# 5. Find the region with the highest average sale price. 
# Determines which regions sell higher-value products.

answer_5= pd.read_sql_query(""" SELECT region, 
ROUND(AVG(sale_price),2) AS avg_sale_price FROM df3
GROUP BY region
ORDER BY avg_sale_price DESC
; """,myconnection)

# 6. Find the total profit per category.
# Identifies the most profitable product categories.

answer_6= pd.read_sql_query(""" SELECT category,
ROUND(SUM(profit),2) AS total_profit FROM df3
GROUP BY category
ORDER BY total_profit;
 """,myconnection)

#  7. Identify the top 3 segments with the highest quantity of orders.
# Highlights which customer segments order the most.

answer_7= pd.read_sql_query(""" SELECT segment, 
SUM(quantity) AS total_quantity FROM df3
GROUP BY segment
ORDER BY total_quantity DESC
LIMIT 3;
 """,myconnection)

# 8. Determine the average discount percentage given per region.
# Analyzes regional discounting strategies.

answer_8= pd.read_sql_query(""" SELECT region, 
CONCAT(ROUND(AVG(discount_percent), 2), "%") AS avg_discount_percent FROM df3
GROUP BY region
ORDER BY avg_discount_percent DESC; """,myconnection)

# 9. Find the product category with the highest total profit.
# Identifies the most profitable category.

answer_9= pd.read_sql_query(""" SELECT category,
ROUND(SUM(profit)) AS total_profit FROM df3
GROUP BY category
ORDER BY total_profit DESC
LIMIT 1;
 """,myconnection)

# 10. Calculate the total revenue generated per year.
# Helps track revenue trends year-over-year.

answer_10= pd.read_sql_query(""" SELECT YEAR(order_date) AS year, 
ROUND(SUM(sale_price * quantity),2) AS total_revenue FROM df3
GROUP BY YEAR(order_date)
ORDER BY year ASC;
  """,myconnection)

# 11. Identify the Regions With the Highest Repeat Orders
# Identifies which regions have the most repeat customers.

answer_11= pd.read_sql_query(""" SELECT region, 
COUNT(DISTINCT order_id) AS total_orders, 
COUNT(order_id) - COUNT(DISTINCT order_id) AS repeat_orders
FROM df3
GROUP BY region
ORDER BY repeat_orders DESC
LIMIT 5;
 """,myconnection)

# 12. Determine the Impact of Discounts on Profitability
#  Helps analyze whether higher discounts lead to lower profitability.

answer_12= pd.read_sql_query(""" SELECT 
    CASE 
        WHEN discount_percent > 0.20 THEN 'High Discount (>20%)'
        ELSE 'Low Discount (≤20%)'
    END AS discount_category,
    ROUND(SUM(profit), 2) AS total_profit,
    ROUND(SUM(sale_price * quantity), 2) AS total_revenue,
    ROUND((SUM(profit) / NULLIF(SUM(sale_price * quantity), 0)) * 100, 2) AS profit_margin
FROM df3 
GROUP BY discount_category;
 """,myconnection)

# 13. Find the Average Order Value (AOV) Per Segment
# Helps in understanding the purchasing power of different segments.

answer_13= pd.read_sql_query(""" SELECT segment, 
ROUND(SUM(sale_price * quantity) / COUNT(DISTINCT order_id), 2) AS avg_order_value
FROM df3 
GROUP BY segment
ORDER BY avg_order_value DESC;
 """,myconnection)

# 14. Identify the Products With the Highest Order Frequency
# Identifies products that are ordered most frequently.

answer_14= pd.read_sql_query(""" SELECT a.product_id, b.sub_category, b.category, 
       COUNT(a.order_id) AS order_count
FROM df3 a
JOIN df2 b ON a.product_id = b.product_id
GROUP BY a.product_id, b.sub_category, b.category
ORDER BY order_count DESC
LIMIT 10;
 """,myconnection)

# 15. Find the Number of Orders Per Region
# Helps analyze which regions have the most orders.

answer_15= pd.read_sql_query(""" SELECT region, 
COUNT(DISTINCT order_id) AS order_count
FROM df3 
GROUP BY region
ORDER BY order_count DESC;
 """,myconnection)

# 16. Find the Month With the Highest Sales
# Helps in identifying seasonal sales trends

answer_16= pd.read_sql_query(""" SELECT EXTRACT(YEAR FROM order_date) AS year, 
EXTRACT(MONTH FROM order_date) AS month, 
ROUND(SUM(sale_price * quantity), 2) AS total_sales
FROM df3 
GROUP BY year, month
ORDER BY total_sales DESC;
 """,myconnection)

# 17. Identify the Top 3 States With the Highest Revenue
#  Helps determine which states contribute most to revenue.

answer_17= pd.read_sql_query(""" SELECT state, 
ROUND(SUM(sale_price * quantity), 2) AS total_revenue
FROM df3 
GROUP BY state
ORDER BY total_revenue DESC
LIMIT 5;
 """,myconnection)

# 18. Calculate the Profit Margin Per Category
# Helps determine which product categories have the best profitability.

answer_18= pd.read_sql_query(""" SELECT b.category, 
ROUND(SUM(a.profit) / NULLIF(SUM(a.sale_price * a.quantity), 0) * 100, 2) AS profit_margin
FROM df3 a
JOIN df2 b ON a.product_id = b.product_id
GROUP BY b.category
ORDER BY profit_margin DESC;
 """,myconnection)

# 19.Identify the Top 5 Most Discounted Products
#  Identifies products that receive the highest average discounts.

answer_19= pd.read_sql_query(""" SELECT a.product_id, b.sub_category, b.category, 
       ROUND(AVG(a.discount_percent) * 100, 2) AS avg_discount_percentage
FROM df3 a
JOIN df2 b ON a.product_id = b.product_id
GROUP BY a.product_id, b.sub_category, b.category
ORDER BY avg_discount_percentage DESC
LIMIT 5;
 """,myconnection)

# 20. Identify the highest revenue-generating customer segment
# Helps identify which customer segments bring the most revenue

answer_20= pd.read_sql_query(""" SELECT segment, 
ROUND(SUM(sale_price * quantity), 2) AS total_revenue
FROM df3 
GROUP BY segment
ORDER BY total_revenue DESC
LIMIT 1;
 """,myconnection)

# 21.Query sales data by region to identify which areas are performing best
# Helps to identify which areas are performing best

answer_21=pd.read_sql_query(""" SELECT region, 
ROUND(SUM(sale_price * quantity),2) AS total_revenue,
ROUND(SUM(profit),2) AS total_profit, 
COUNT(DISTINCT order_id) AS order_count FROM df3
GROUP BY region
ORDER BY total_revenue DESC;"""
,myconnection)

# 22.Compare year-over-year sales to identify growth or decline in certain months
# Helps to identify the growth in Year-over-year(month) sales trend

answer_22=pd.read_sql_query(""" 
SELECT * 
FROM (
    SELECT
        EXTRACT(YEAR FROM order_date) AS year,
        EXTRACT(MONTH FROM order_date) AS month,
        ROUND(SUM(sale_price * quantity), 2) AS total_sales,
        LAG(ROUND(SUM(sale_price * quantity), 2)) OVER (
            PARTITION BY EXTRACT(MONTH FROM order_date) 
            ORDER BY EXTRACT(YEAR FROM order_date)
        ) AS previous_sales,
        ROUND(
        	((SUM(sale_price * quantity) - 
        	LAG(SUM(sale_price * quantity)) OVER (
            	PARTITION BY EXTRACT(MONTH FROM order_date) 
        		ORDER BY EXTRACT(YEAR FROM order_date)
        	)) / NULLIF(LAG(SUM(sale_price * quantity)) OVER (
        		PARTITION BY EXTRACT(MONTH FROM order_date) 
        		ORDER BY EXTRACT(YEAR FROM order_date)
    		), 0)) * 100,2) AS year_growth
    FROM df3
    GROUP BY year, month
) AS yearsales
WHERE year = 2023
ORDER BY month, year; """,myconnection)


In [10]:
# 1. Find top 10 highest revenue generating products.
# Identifies best-selling products that drive the most revenue.

answer_1 = pd.read_sql_query("""SELECT b.category,b.sub_category,a.product_id, 
ROUND(SUM(a.sale_price * a.quantity), 2) AS revenue FROM df3 a
JOIN df2 b ON a.order_id = b.order_id
GROUP BY b.category, b.sub_category, a.product_id
ORDER BY revenue DESC
LIMIT 10;
""",myconnection)

answer_1

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2213221221.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_1 = pd.read_sql_query("""SELECT b.category,b.sub_category,a.product_id,


,category,sub_category,product_id,revenue
0,Technology,Copiers,TEC-CO-10004722,245056.0
1,Office Supplies,Binders,OFF-BI-10000545,163777.7
2,Technology,Machines,TEC-MA-10002412,130406.4
3,Furniture,Chairs,FUR-CH-10002024,120090.7
4,Technology,Phones,TEC-PH-10001459,113041.9
5,Technology,Copiers,TEC-CO-10001449,107388.0
6,Office Supplies,Binders,OFF-BI-10003527,97082.9
7,Technology,Machines,TEC-MA-10000822,89622.3
8,Furniture,Bookcases,FUR-BO-10002213,84014.8
9,Technology,Machines,TEC-MA-10001047,81549.0


In [11]:
# 2. Find the top 5 cities with the highest profit margins.
# Helps understand which cities yield the highest profit margins.

answer_2 = pd.read_sql_query("""SELECT b.city, ROUND((SUM(a.profit) / SUM(a.sale_price * a.quantity)) * 100, 2) AS profit_margin FROM df3 a
JOIN df1 b ON a.order_id = b.order_id
GROUP BY b.city
ORDER BY profit_margin DESC
LIMIT 5;
""",myconnection)

answer_2

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\931729614.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_2 = pd.read_sql_query("""SELECT b.city, ROUND((SUM(a.profit) / SUM(a.sale_price * a.quantity)) * 100, 2) AS profit_margin FROM df3 a


,city,profit_margin
0,Danbury,29.82
1,Goldsboro,29.82
2,Bozeman,25.02
3,Rogers,23.47
4,Owensboro,22.87


In [12]:
# 3. Calculate the total discount given for each category.
# Reveals which product categories receive the highest total discounts.

answer_3= pd.read_sql_query(""" SELECT category, 
ROUND(SUM(discount),2) AS total_discount FROM df3
GROUP BY category
ORDER BY total_discount DESC;
;
 """,myconnection)

answer_3

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\3040603334.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_3= pd.read_sql_query(""" SELECT category,


,category,total_discount
0,Technology,29319.2
1,Furniture,25996.5
2,Office Supplies,24923.6


In [13]:
# 4. Find the average sale price per product category.
# Helps in pricing strategy and competitive analysis.

answer_4= pd.read_sql_query(""" SELECT category, 
ROUND(AVG(sale_price),2) AS avg_sale_price FROM df3
GROUP BY category
ORDER BY avg_sale_price DESC;
""",myconnection)

answer_4

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2081107648.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_4= pd.read_sql_query(""" SELECT category,


,category,avg_sale_price
0,Technology,436.61
1,Furniture,336.78
2,Office Supplies,115.13


In [14]:
# 5. Find the region with the highest average sale price. 
# Determines which regions sell higher-value products.

answer_5= pd.read_sql_query(""" SELECT region, 
ROUND(AVG(sale_price),2) AS avg_sale_price FROM df3
GROUP BY region
ORDER BY avg_sale_price DESC
; """,myconnection)

answer_5

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\391632379.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_5= pd.read_sql_query(""" SELECT region,


,region,avg_sale_price
0,South,233.39
1,East,229.75
2,West,217.96
3,Central,208.27


In [15]:
# 6. Find the total profit per category.
# Identifies the most profitable product categories.

answer_6= pd.read_sql_query(""" SELECT category,
ROUND(SUM(profit),2) AS total_profit FROM df3
GROUP BY category
ORDER BY total_profit;
 """,myconnection)

answer_6

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2653181140.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_6= pd.read_sql_query(""" SELECT category,


,category,total_profit
0,Office Supplies,317381.8
1,Furniture,341094.6
2,Technology,379189.9


In [16]:
#  7. Identify the top 3 segments with the highest quantity of orders.
# Highlights which customer segments order the most.

answer_7= pd.read_sql_query(""" SELECT segment, 
SUM(quantity) AS total_quantity FROM df3
GROUP BY segment
ORDER BY total_quantity DESC
LIMIT 3;
 """,myconnection)

answer_7

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\1203381734.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_7= pd.read_sql_query(""" SELECT segment,


,segment,total_quantity
0,Consumer,19498.0
1,Corporate,11607.0
2,Home Office,6739.0


In [17]:
# 8. Determine the average discount percentage given per region.
# Analyzes regional discounting strategies.

answer_8= pd.read_sql_query(""" SELECT region, 
CONCAT(ROUND(AVG(discount_percent), 2), "%") AS avg_discount_percent FROM df3
GROUP BY region
ORDER BY avg_discount_percent DESC; """,myconnection)

answer_8

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2467457301.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_8= pd.read_sql_query(""" SELECT region,


,region,avg_discount_percent
0,West,3.49%
1,East,3.49%
2,South,3.48%
3,Central,3.47%


In [18]:
# 9. Find the product category with the highest total profit.
# Identifies the most profitable category.

answer_9= pd.read_sql_query(""" SELECT category,
ROUND(SUM(profit)) AS total_profit FROM df3
GROUP BY category
ORDER BY total_profit DESC
LIMIT 1;
 """,myconnection)

answer_9

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2109088195.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_9= pd.read_sql_query(""" SELECT category,


,category,total_profit
0,Technology,379190.0


In [19]:
# 10. Calculate the total revenue generated per year.
# Helps track revenue trends year-over-year.

answer_10= pd.read_sql_query(""" SELECT YEAR(order_date) AS year, 
ROUND(SUM(sale_price * quantity),2) AS total_revenue FROM df3
GROUP BY YEAR(order_date)
ORDER BY year ASC;
  """,myconnection)

answer_10

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2728016456.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_10= pd.read_sql_query(""" SELECT YEAR(order_date) AS year,


,year,total_revenue
0,2022,5475201.8
1,2023,5584784.5


In [20]:
# 11. Identify the Regions With the Highest Repeat Orders
# Identifies which regions have the most repeat customers.

answer_11= pd.read_sql_query(""" SELECT region, 
COUNT(DISTINCT order_id) AS total_orders, 
COUNT(order_id) - COUNT(DISTINCT order_id) AS repeat_orders
FROM df3
GROUP BY region
ORDER BY repeat_orders DESC
LIMIT 5;
 """,myconnection)

answer_11

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\4189040072.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_11= pd.read_sql_query(""" SELECT region,


,region,total_orders,repeat_orders
0,Central,2322,0
1,East,2848,0
2,South,1619,0
3,West,3199,0


In [21]:
# 12. Determine the Impact of Discounts on Profitability
#  Helps analyze whether higher discounts lead to lower profitability.

answer_12= pd.read_sql_query(""" SELECT 
    CASE 
        WHEN discount_percent > 0.20 THEN 'High Discount (>20%)'
        ELSE 'Low Discount (≤20%)'
    END AS discount_category,
    ROUND(SUM(profit), 2) AS total_profit,
    ROUND(SUM(sale_price * quantity), 2) AS total_revenue,
    ROUND((SUM(profit) / NULLIF(SUM(sale_price * quantity), 0)) * 100, 2) AS profit_margin
FROM df3 
GROUP BY discount_category;
 """,myconnection)

answer_12

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\3261985358.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_12= pd.read_sql_query(""" SELECT


,discount_category,total_profit,total_revenue,profit_margin
0,High Discount (>20%),1037666.3,11059986.29,9.38


In [22]:
# 13. Find the Average Order Value (AOV) Per Segment
# Helps in understanding the purchasing power of different segments.

answer_13= pd.read_sql_query(""" SELECT segment, 
ROUND(SUM(sale_price * quantity) / COUNT(DISTINCT order_id), 2) AS avg_order_value
FROM df3 
GROUP BY segment
ORDER BY avg_order_value DESC;
 """,myconnection)

answer_13

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2766408421.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_13= pd.read_sql_query(""" SELECT segment,


,segment,avg_order_value
0,Home Office,1165.11
1,Corporate,1124.47
2,Consumer,1077.50


In [23]:
# 14. Identify the Products With the Highest Order Frequency
# Identifies products that are ordered most frequently.

answer_14= pd.read_sql_query(""" SELECT a.product_id, b.sub_category, b.category, 
       COUNT(DISTINCT a.order_id) AS order_count
FROM df3 a
JOIN df2 b ON a.product_id = b.product_id
GROUP BY a.product_id, b.sub_category, b.category
ORDER BY order_count DESC
LIMIT 10;
 """,myconnection)

answer_14

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\3910592662.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_14= pd.read_sql_query(""" SELECT a.product_id, b.sub_category, b.category,


,product_id,sub_category,category,order_count
0,OFF-PA-10001970,Paper,Office Supplies,19
1,TEC-AC-10003832,Accessories,Technology,18
2,FUR-FU-10004270,Furnishings,Furniture,16
3,TEC-AC-10002049,Accessories,Technology,15
4,FUR-CH-10001146,Chairs,Furniture,15
5,TEC-AC-10003628,Accessories,Technology,15
6,FUR-CH-10002647,Chairs,Furniture,15
7,OFF-BI-10004632,Binders,Office Supplies,14
8,FUR-FU-10001473,Furnishings,Furniture,14
9,FUR-CH-10003774,Chairs,Furniture,14


In [24]:
# 15. Find the Number of Orders Per Region
# Helps analyze which regions have the most orders.

answer_15= pd.read_sql_query(""" SELECT region, 
COUNT(DISTINCT order_id) AS order_count
FROM df3 
GROUP BY region
ORDER BY order_count DESC;
 """,myconnection)

answer_15

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\3168502616.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_15= pd.read_sql_query(""" SELECT region,


,region,order_count
0,West,3199
1,East,2848
2,Central,2322
3,South,1619


In [25]:
# 16. Find the Month With the Highest Sales
# Helps in identifying seasonal sales trends

answer_16= pd.read_sql_query(""" SELECT EXTRACT(YEAR FROM order_date) AS year, 
EXTRACT(MONTH FROM order_date) AS month, 
ROUND(SUM(sale_price * quantity), 2) AS total_sales
FROM df3 
GROUP BY year, month
ORDER BY total_sales DESC;
 """,myconnection)

answer_16

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\2428742692.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_16= pd.read_sql_query(""" SELECT EXTRACT(YEAR FROM order_date) AS year,


,year,month,total_sales
0,2023,2,731638.8
1,2023,10,626498.3
2,2022,10,601707.8
3,2023,4,543231.5
4,2022,8,534562.4
5,2023,12,488318.1
6,2022,4,476400.9
7,2022,6,465300.3
8,2023,8,465010.3
9,2022,11,451809.6


In [26]:
# 17. Identify the Top 3 States With the Highest Revenue
#  Helps determine which states contribute most to revenue.

answer_17= pd.read_sql_query(""" SELECT state, 
ROUND(SUM(sale_price * quantity), 2) AS total_revenue
FROM df3 
GROUP BY state
ORDER BY total_revenue DESC
LIMIT 5;
 """,myconnection)

answer_17

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\672305576.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_17= pd.read_sql_query(""" SELECT state,


,state,total_revenue
0,California,2200661.7
1,New York,1503857.1
2,Texas,804546.7
3,Washington,668161.2
4,Pennsylvania,581119.1


In [27]:
# 18. Calculate the Profit Margin Per Category
# Helps determine which product categories have the best profitability.

answer_18= pd.read_sql_query(""" SELECT b.category, 
ROUND(SUM(a.profit) / NULLIF(SUM(a.sale_price * a.quantity), 0) * 100, 2) AS profit_margin
FROM df3 a
JOIN df2 b ON a.product_id = b.product_id
GROUP BY b.category
ORDER BY profit_margin DESC;
 """,myconnection)

answer_18

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\1658808286.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_18= pd.read_sql_query(""" SELECT b.category,


,category,profit_margin
0,Technology,9.54
1,Office Supplies,9.30
2,Furniture,9.13


In [28]:
# 19.Identify the Most Discounted Products
#  Identifies products that receive the highest average discounts.

answer_19= pd.read_sql_query(""" SELECT a.product_id, b.sub_category, b.category, 
       ROUND(AVG(a.discount_percent) * 100, 2) AS avg_discount_percentage
FROM df3 a
JOIN df2 b ON a.product_id = b.product_id
GROUP BY a.product_id, b.sub_category, b.category
ORDER BY avg_discount_percentage DESC;
 """,myconnection)

answer_19

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\1790381741.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_19= pd.read_sql_query(""" SELECT a.product_id, b.sub_category, b.category,


,product_id,sub_category,category,avg_discount_percentage
0,OFF-AP-10000326,Appliances,Office Supplies,500.0
1,OFF-ST-10001414,Storage,Office Supplies,500.0
2,TEC-AC-10000290,Accessories,Technology,500.0
3,OFF-AP-10002495,Appliances,Office Supplies,500.0
4,FUR-BO-10001918,Bookcases,Furniture,500.0
...,...,...,...,...
1857,TEC-MA-10003673,Machines,Technology,200.0
1858,TEC-MA-10001695,Machines,Technology,200.0
1859,TEC-MA-10004626,Machines,Technology,200.0
1860,OFF-AR-10003477,Art,Office Supplies,200.0


In [29]:
# 20. Find the Customer Segments Generating the Highest Revenue
# Helps identify which customer segments bring the most revenue

answer_20= pd.read_sql_query(""" SELECT segment, 
ROUND(SUM(sale_price * quantity), 2) AS total_revenue
FROM df3 
GROUP BY segment
ORDER BY total_revenue DESC
LIMIT 1;
 """,myconnection)

answer_20

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\914658182.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_20= pd.read_sql_query(""" SELECT segment,


,segment,total_revenue
0,Consumer,5589002.7


In [30]:
# 21.Query sales data by region to identify which areas are performing best
# Helps to identify which areas are performing best

answer_21=pd.read_sql_query(""" SELECT region, 
ROUND(SUM(sale_price * quantity),2) AS total_revenue,
ROUND(SUM(profit),2) AS total_profit, 
COUNT(DISTINCT order_id) AS order_count FROM df3
GROUP BY region
ORDER BY total_revenue DESC;"""
,myconnection)

answer_21

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\500606349.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_21=pd.read_sql_query(""" SELECT region,


,region,total_revenue,total_profit,order_count
0,West,3448552.19,303072.2,3199
1,East,3257983.80,295773.8,2848
2,Central,2387548.70,236088.7,2322
3,South,1965901.60,202731.6,1619


In [31]:
# 22.Compare year-over-year sales to identify growth or decline in certain months
# Helps to identify the growth in Year-over-year(month) sales trend

answer_22=pd.read_sql_query(""" 
SELECT * 
FROM (
    SELECT
        EXTRACT(YEAR FROM order_date) AS year,
        EXTRACT(MONTH FROM order_date) AS month,
        ROUND(SUM(sale_price * quantity), 2) AS total_sales,
        LAG(ROUND(SUM(sale_price * quantity), 2)) OVER (
            PARTITION BY EXTRACT(MONTH FROM order_date) 
            ORDER BY EXTRACT(YEAR FROM order_date)
        ) AS previous_sales,
        ROUND(
        	((SUM(sale_price * quantity) - 
        	LAG(SUM(sale_price * quantity)) OVER (
            	PARTITION BY EXTRACT(MONTH FROM order_date) 
        		ORDER BY EXTRACT(YEAR FROM order_date)
        	)) / NULLIF(LAG(SUM(sale_price * quantity)) OVER (
        		PARTITION BY EXTRACT(MONTH FROM order_date) 
        		ORDER BY EXTRACT(YEAR FROM order_date)
    		), 0)) * 100,2) AS year_growth
    FROM df3
    GROUP BY year, month
) AS yearsales
WHERE year = 2023
ORDER BY month, year; """,myconnection)

answer_22

C:\Users\Admin\AppData\Local\Temp\ipykernel_6480\867052259.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_22=pd.read_sql_query("""


,year,month,total_sales,previous_sales,year_growth
0,2023,1,434765.5,437431.3,-0.61
1,2023,2,731638.8,444011.1,64.78
2,2023,3,378064.8,393765.7,-3.99
3,2023,4,543231.5,476400.9,14.03
4,2023,5,410707.9,413625.5,-0.71
5,2023,6,328939.0,465300.3,-29.31
6,2023,7,422381.7,375278.4,12.55
7,2023,8,465010.3,534562.4,-13.01
8,2023,9,420620.5,433887.0,-3.06
9,2023,10,626498.3,601707.8,4.12
